In [42]:
pip install tensorflow==1.13

  Could not find a version that satisfies the requirement tensorflow==1.13 (from versions: 1.13.0rc1, 1.13.0rc2, 1.13.1, 1.13.2, 1.14.0rc0, 1.14.0rc1, 1.14.0, 1.15.0rc0, 1.15.0rc1, 1.15.0rc2, 1.15.0rc3, 1.15.0, 1.15.2, 1.15.3, 1.15.4, 2.0.0a0, 2.0.0b0, 2.0.0b1, 2.0.0rc0, 2.0.0rc1, 2.0.0rc2, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.1.0rc0, 2.1.0rc1, 2.1.0rc2, 2.1.0, 2.1.1, 2.1.2, 2.2.0rc0, 2.2.0rc1, 2.2.0rc2, 2.2.0rc3, 2.2.0rc4, 2.2.0, 2.2.1, 2.3.0rc0, 2.3.0rc1, 2.3.0rc2, 2.3.0, 2.3.1)
No matching distribution found for tensorflow==1.13
Note: you may need to restart the kernel to use updated packages.


In [21]:
# import dependencies
import numpy
import sys
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
# load data
# Project Gutenberg/ berg where frankenstein dataset can be found
file = open("Mahabharat.txt").read()

In [23]:
# tokenization
# standardization
def tokenize_words(input):
    input = input.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)
    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return "".join(filtered)

processed_inputs = tokenize_words(file)

In [25]:
# chars to numbers
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c,i) for i, c in enumerate(chars))

In [26]:
# check if words to chars to num (?!) has worked?
input_len = len(processed_inputs)
vocab_len = len(chars)
print("Total number of characters: ", input_len)
print("Total vocab: ", vocab_len)

Total number of characters:  341895
Total vocab:  27


In [27]:
# seq length
seq_length = 100
x_data = []
y_data = []

In [28]:
# loop through the sequence
for i in range(0, input_len - seq_length, 1):
    in_seq = processed_inputs[i:i + seq_length]
    out_seq = processed_inputs[i + seq_length]
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])
    
n_patterns = len(x_data)
print("Total Patterns:", n_patterns)

Total Patterns: 341795


In [29]:
# convert input sequence to np array and so on
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [30]:
# one-hot encoding
y = np_utils.to_categorical(y_data)

In [31]:
# creating the model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))

In [32]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [34]:
# saving weights
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose = 1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [35]:
# fit the model and let it train
model.fit(X,y, epochs=4, batch_size=256, callbacks=desired_callbacks)

Epoch 1/4
1336/1336 [==============================] - ETA: 0s - loss: 2.9256
Epoch 00001: loss improved from inf to 2.92563, saving model to model_weights_saved.hdf5
1336/1336 [==============================] - 219s 164ms/step - loss: 2.9256
Epoch 2/4
1336/1336 [==============================] - ETA: 0s - loss: 2.8684
Epoch 00002: loss improved from 2.92563 to 2.86838, saving model to model_weights_saved.hdf5
1336/1336 [==============================] - 218s 163ms/step - loss: 2.8684
Epoch 3/4
1336/1336 [==============================] - ETA: 0s - loss: 2.8038
Epoch 00003: loss improved from 2.86838 to 2.80378, saving model to model_weights_saved.hdf5
1336/1336 [==============================] - 218s 163ms/step - loss: 2.8038
Epoch 4/4
1336/1336 [==============================] - ETA: 0s - loss: 2.7284
Epoch 00004: loss improved from 2.80378 to 2.72835, saving model to model_weights_saved.hdf5
1336/1336 [==============================] - 218s 163ms/step - loss: 2.7284


In [36]:
# recompile model with the saved weights
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [37]:
# output of the model back into charcters
num_to_char = dict((i,c) for i,c in enumerate(chars))

In [38]:
# random seed to help generate
start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
" lskilledestablishingpropositionsrejectingsuperfluouscausesdrawingrightconclusionsknowledgesciencewor "


In [39]:
# generate the text
for i in range(1000):
    x = numpy.reshape(pattern, (1,len(pattern), 1))
    x = x/float(vocab_len)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = num_to_char[index]
    seq_in = [num_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

eserseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseateeterseatee